# Dataset Creator

## How to get data

1. Data taken from http://www.naturalstattrick.com/

2. Go in players -> individiual

3. Use these settings

<img src="../figures/data-ex.png" alt="alt" style="float: left; margin: 20px;" />


4. Click on `CSV (all)` and save under the name `nhl-19-20-ind-rates.csv`

5. Change from `All Strengths` to `Power Play`, click on `CSV (all)` and save under the name `nhl-19-20-ind-pp-rates.csv`

6. Change from `Individual` to `On Ice`, click on `CSV (all)` and save under the name `nhl-19-20-ice-pp-rates.csv`

7. Change from `Power Play` to `All Strengths`, click on `CSV (all)` and save under the name `nhl-19-20-ice-rates.csv`

8. Change from `All Strengths` to `Power Play` and `Rates` to `Counts`, click on `CSV (all)` and save under the name `nhl-19-20-ind-pp.csv`

9. Go in players -> bios

10. Use these settings

<img src="../figures/data-bio.png" alt="alt" style="float: left; margin: 20px;" />
     
11. Click on `CSV (all)` and save under the name `nhl-19-20-bio.csv`

12. Put the data in `../data/nhl-stats_rates`

13. Download goalie data from hockey reference (https://www.hockey-reference.com/leagues/NHL_2020_goalies.html)

## Note

* CTRL-F and search for `TODO` to modify parts dependant of the nhl season.

* Yearly projections are made by a redditor `u/ChuckLemieux`, watch out for his google spreadsheet. It aggregates a bunch of projections.

## 1. Preparation
---

In [1]:
import pandas as pd
import numpy as np
import re
import math
import pickle
import glob
import xgboost
import tsfresh
import os
from pprint import pprint
from sklearn import svm
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression, SGDRegressor, Ridge
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.tree import export_graphviz
from sklearn.metrics import mean_absolute_error, mean_squared_error, explained_variance_score, r2_score, log_loss
from sklearn.model_selection import cross_validate, cross_val_predict
from sklearn import preprocessing
from sklearn.model_selection import cross_val_score
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

sns.set_context('notebook')
sns.set_color_codes()
plot_kwds = {'alpha' : 0.5, 's' : 100, 'linewidths':0}
large = 22; med = 16; small = 12
params = {'axes.titlesize': large,
          'legend.fontsize': med,
          'figure.figsize': (16, 10),
          'axes.labelsize': med,
          'axes.titlesize': med,
          'xtick.labelsize': med,
          'ytick.labelsize': med,
          'figure.titlesize': large}
plt.rcParams.update(params)
plt.style.use('seaborn-whitegrid')
sns.set_style("white")

pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

pd.options.mode.chained_assignment = None  # default='warn'

In [2]:
# Paths
data_folder = '../data/'

## 2. Read datasets
---

In [21]:
def read_dataset(file_paths_regex):
    data_paths = glob.glob(file_paths_regex)
    data_paths.sort()
    
    data = pd.DataFrame()
    for file_path in data_paths:
        file = pd.read_csv(file_path, sep=',', encoding="utf-8-sig")
        file['Year'] = '20' + file_path.split('-')[2]
        file = file.rename(columns={'Unnamed: 0': 'Index'}).set_index('Index')
        data = pd.concat([data, file], sort=False)
    data['Name'] = data.apply(lambda player: re.sub(r'\W+', '', f'{player["Player"]})').upper(), axis=1)
    data.set_index('Name', inplace=True)
    return data

In [69]:
# Read the 3 kinds of datasets
individuals = read_dataset(os.path.join(data_folder, 'nhl-stats_rates/*ind-rates.csv'))
on_ice = read_dataset(os.path.join(data_folder, './nhl-stats_rates/*ice-rates.csv'))
on_ice.drop(['GP', 'TOI'], axis=1, inplace=True) # Because already in individuals
on_ice_pp = read_dataset(os.path.join(data_folder, './nhl-stats_rates/*ind-pp.csv'))
on_ice_pp.drop(['GP'], axis=1, inplace=True) # Because already in individuals
new_cols = ['_'.join([col, 'PP']) for col in on_ice_pp.columns[3:]]
new_cols = ['Player', 'Team', 'Position'] + new_cols
new_cols[-1] = 'Year'
on_ice_pp.columns = new_cols
biography = read_dataset(os.path.join(data_folder, './nhl-stats_rates/*bio.csv'))

# Merge the 3 datasets together
tmp = pd.merge(biography, individuals, on=['Player', 'Team', 'Position', 'Year'])
tmp = pd.merge(tmp, on_ice, on=['Player', 'Team', 'Position', 'Year'])
nhl_players = pd.merge(tmp, on_ice_pp, on=['Player', 'Team', 'Position', 'Year'])

# List of columns
cols = list(nhl_players)
                                    
# Move the column to head of list using index, pop and insert Year first
cols.insert(0, cols.pop(cols.index('Year')))

# Reorder
nhl_players = nhl_players.loc[:, cols]
nhl_players = nhl_players.sort_values(['Player', 'Year'])

nhl_players.head(10)

,Year,Player,Team,Position,Age,Date of Birth,Birth City,Birth State/Province,Birth Country,Nationality,Height (in),Weight (lbs),Draft Year,Draft Team,Draft Round,Round Pick,Overall Draft Position,GP,TOI,TOI/GP_x,Goals/60,Total Assists/60,First Assists/60,Second Assists/60,Total Points/60,IPP,Shots/60,SH%,iCF/60,iFF/60,iSCF/60,iHDCF/60,Rush Attempts/60,Rebounds Created/60,PIM/60,Total Penalties/60,Minor/60,Major/60,Misconduct/60,Penalties Drawn/60,Giveaways/60,Takeaways/60,Hits/60,Hits Taken/60,Shots Blocked/60,Faceoffs Won/60,Faceoffs Lost/60,Faceoffs %,ixG/60,TOI/GP_y,CF/60,CA/60,CF%,FF/60,FA/60,FF%,SF/60,SA/60,SF%,GF/60,GA/60,GF%,SCF/60,SCA/60,SCF%,HDCF/60,HDCA/60,HDCF%,HDGF/60,HDGA/60,HDGF%,MDCF/60,MDCA/60,MDCF%,MDGF/60,MDGA/60,MDGF%,LDCF/60,LDCA/60,LDCF%,LDGF/60,LDGA/60,LDGF%,On-Ice SH%,On-Ice SV%,PDO,Off. Zone Starts/60,Neu. Zone Starts/60,Def. Zone Starts/60,On The Fly Starts/60,Off. Zone Start %,Off. Zone Faceoffs/60,Neu. Zone Faceoffs/60,Def. Zone Faceoffs/60,Off. Zone Faceoff %,xGF/60,xGA/60,xGF%,TOI_PP,Goals_PP,Total Assists_PP,First Assists_PP,Second Assists_PP,Total Points_PP,IPP_PP,Shots_PP,SH%_PP,ixG_PP,iCF_PP,iFF_PP,iSCF_PP,iHDCF_PP,Rush Attempts_PP,Rebounds Created_PP,PIM_PP,Total Penalties_PP,Minor_PP,Major_PP,Misconduct_PP,Penalties Drawn_PP,Giveaways_PP,Takeaways_PP,Hits_PP,Hits Taken_PP,Shots Blocked_PP,Faceoffs Won_PP,Faceoffs Lost_PP,Faceoffs %_PP
1630,2016,A.J. Greer,COL,L,21,1996-12-14,Joliette,QC,CAN,CAN,75,210,2015,COL,2,9,39,5,68.283333,13.656667,0.00,0.88,0.00,0.88,0.88,100.00,7.03,0.00,9.67,8.79,6.15,3.51,0.00,1.76,3.51,1.76,1.76,0.00,0.00,1.76,1.76,0.88,3.51,5.27,2.64,0.00,0.00,-,NaN,13.656667,57.99,50.09,53.66,44.81,33.39,57.3,31.63,28.12,52.94,0.88,2.64,25.00,27.24,12.30,68.89,13.18,3.51,78.95,0.88,0.00,100.00,18.75,11.72,61.54,0.00,2.34,0.00,36.81,39.35,48.33,0.00,1.27,0.00,2.78,90.63,0.934,11.42,20.21,12.30,45.69,48.15,28.12,27.24,25.48,52.46,NaN,NaN,NaN,2.866667,0,0,0,0,0,-,2,0.00,0.42,2,2,2,1,0,1,2,1,1,0,0,0,0,0,0,0,0,0,0,-
2441,2017,A.J. Greer,COL,L,21,1996-12-14,Joliette,QC,CAN,CAN,75,210,2015,COL,2,9,39,17,125.683333,7.393137,0.00,1.43,0.95,0.48,1.43,75.00,6.21,0.00,9.55,7.16,3.82,1.43,0.48,0.00,13.84,4.30,3.34,0.48,0.48,3.34,0.95,1.43,14.32,7.64,2.39,0.48,0.95,15.91,NaN,7.393137,42.01,61.58,40.55,32.46,47.74,40.48,25.78,34.85,42.52,1.91,0.95,66.67,21.96,25.78,46,8.12,12.41,39.53,0.95,0.48,66.67,18.46,17.82,50.88,0.64,0.00,100.00,24.13,41.37,36.84,0.69,0.69,50.00,7.41,97.26,1.047,11.46,10.03,10.03,58.24,53.33,17.19,16.23,15.75,52.17,NaN,NaN,NaN,2.783333,0,0,0,0,0,-,0,-,0.00,0,0,0,0,0,0,7,2,1,1,0,2,0,0,1,0,0,0,0,-
0,2014,Aaron Ekblad,FLA,D,22,1996-02-07,Windsor,ON,CAN,CAN,76,220,2014,FLA,1,1,1,81,1766.616667,21.810082,0.41,0.92,0.34,0.58,1.32,44.32,5.77,7.06,11.92,8.08,2.65,0.48,0.17,0.95,1.09,0.54,0.54,0.00,0.00,0.51,1.39,0.88,3.70,4.62,2.72,0.00,0.00,-,NaN,21.810082,59.84,44.02,57.62,46.16,34.57,57.17,35.32,25.17,58.39,2.99,1.97,60.27,26.83,19.05,58.48,10.05,8.02,55.64,1.46,0.98,59.72,22.37,14.72,60.32,1.13,0.72,60.98,41.94,30.81,57.65,0.98,0.54,64.52,8.46,92.17,1.006,11.65,10.32,5.26,38.55,68.88,26.76,17.29,13.69,66.16,NaN,NaN,NaN,226.550000,6,7,1,6,13,59.09,39,15.38,3.49,83,58,19,5,0,7,4,2,2,0,0,0,2,3,6,6,2,0,0,-
803,2015,Aaron Ekblad,FLA,D,22,1996-02-07,Windsor,ON,CAN,CAN,76,220,2014,FLA,1,1,1,78,1690.816667,21.677137,0.53,0.75,0.28,0.46,1.28,38.71,6.46,8.24,11.50,8.52,3.05,0.82,0.18,0.57,1.45,0.67,0.64,0.04,0.00,0.32,1.45,0.85,3.09,4.47,2.09,0.00,0.00,-,NaN,21.677137,54.01,43.61,55.33,42.16,34.53,54.97,31.23,24.49,56.05,3.30,2.13,60.78,25.76,19.73,56.63,10.57,8.41,55.70,1.70,1.21,58.54,20.25,15.09,57.30,1.32,0.80,62.22,36.80,29.22,55.75,0.72,0.31,70.00,10.57,91.30,1.019,11.75,13.02,7.77,32.93,60.18,24.20,21.01,15.79,60.51,NaN,NaN,NaN,221.533333,3,6,2,4,9,39.13,38,7.89,3.42,79,53,20,5,0,5,4,2,2,0,0,0,3,1,5,9,1,0,0,-
1631,2016,Aaron Ekblad,FLA,D,22,1996-02-07,Windsor,ON,CAN,CAN,76,220,2014,FLA,1,1,1,68,1459.283333,21.460049,0.41,0.45,0.16,0.29,0.86,38.89,9.25,4.44,16.78,12.17,4.56,0.66

## 3. Preprocessing
---

In [70]:
def get_draft_year(player):
    if player['Draft Year'] == '-':
        return int(player['Date of Birth'].split('-')[0]) + 18
    else:
        return player['Draft Year']
    
def get_draft_position(player):
    if player['Overall Draft Position'] == '-':
        return 0
    else:
        return player['Overall Draft Position']   

### 3.1 Clean Data

In [71]:
# Fix column names
# There is some weird characters for some column names
cols = nhl_players.columns
cols = cols.map(lambda x: x.replace(u'\xa0', u' ') if isinstance(x, (str)) else x)
nhl_players.columns = cols

In [72]:
# Fill missing data
# Age infered based on birth year and current year of season played
nhl_players['Date of Birth'] = nhl_players['Date of Birth'].replace({'-': '2000-01-01'})

# This should not be needed, weird...
# nhl_players['Age'] = nhl_players.apply(lambda plyr: int(plyr['Year'])-int(plyr['Date of Birth'].split('-')[0]), axis=1)
nhl_players['Age'] = nhl_players['Age'].astype(int)

# If not draft year, infered based on date of birth + 18 years
nhl_players['Draft Year'] = nhl_players.apply(lambda plyr: get_draft_year(plyr), axis=1)

# If no overall draft position, we return 0
nhl_players['Overall Draft Position'] = nhl_players.apply(lambda plyr: get_draft_position(plyr), axis=1)

# Real total points
nhl_players['Total Points'] = (nhl_players['Total Points/60'] * nhl_players['TOI'] / 60).astype(int)
nhl_players['Total Points/GP'] = nhl_players['Total Points']/nhl_players['GP']

# If no value is given for a particular stat, we insert the mean of that stat
for col in ['GP', 'TOI', 'Goals/60', 'Total Assists/60', 'First Assists/60', 'Second Assists/60', 'Total Points/60', 
            'IPP', 'Shots/60', 'SH%', 'iCF/60', 'iFF/60', 'iSCF/60', 'iHDCF/60', 'Rush Attempts/60', 'Rebounds Created/60', 
            'PIM/60', 'Total Penalties/60', 'Minor/60', 'Major/60', 'Misconduct/60', 'Penalties Drawn/60', 'Giveaways/60', 
            'Takeaways/60', 'Hits/60', 'Hits Taken/60', 'Shots Blocked/60', 'Faceoffs Won/60', 'Faceoffs Lost/60', 
            'Faceoffs %', 'CF/60', 'CA/60', 'CF%', 'FF/60', 'FA/60', 'FF%', 'SF/60', 'SA/60', 'SF%', 'GF/60', 'GA/60', 'GF%', 
            'SCF/60', 'SCA/60', 'SCF%', 'HDCF/60', 'HDCA/60', 'HDCF%', 'HDGF/60', 'HDGA/60', 'HDGF%', 'MDCF/60', 'MDCA/60', 
            'MDCF%', 'MDGF/60', 'MDGA/60', 'MDGF%', 'LDCF/60', 'LDCA/60', 'LDCF%', 'LDGF/60', 'LDGA/60', 'LDGF%', 
            'On-Ice SH%', 'On-Ice SV%', 'PDO', 'Off. Zone Starts/60',
            'Neu. Zone Starts/60', 'Def. Zone Starts/60', 'On The Fly Starts/60',
            'Off. Zone Start %', 'Off. Zone Faceoffs/60', 'Neu. Zone Faceoffs/60',
            'Def. Zone Faceoffs/60', 'Off. Zone Faceoff %']:
    mean = nhl_players[~nhl_players[col].isin(['-'])][col].astype(float).mean()
    nhl_players[col].replace('-', 0, inplace=True)
    
for col in ['TOI_PP', 'Goals_PP', 'Total Assists_PP', 'First Assists_PP', 'Second Assists_PP', 'Total Points_PP', 
            'IPP_PP', 'Shots_PP', 'SH%_PP', 'iCF_PP', 'iFF_PP', 'iSCF_PP', 'iHDCF_PP', 'Rush Attempts_PP', 
            'Rebounds Created_PP', 'PIM_PP', 'Total Penalties_PP', 'Minor_PP', 'Major_PP', 'Misconduct_PP', 'Penalties Drawn_PP', 
            'Giveaways_PP', 'Takeaways_PP', 'Hits_PP', 'Hits Taken_PP', 'Shots Blocked_PP', 'Faceoffs Won_PP', 'Faceoffs Lost_PP', 
            'Faceoffs %_PP']:
    mean = nhl_players[~nhl_players[col].isin(['-'])][col].astype(float).mean()
    nhl_players[col].replace('-', 0, inplace=True)
    
# To numeric
nhl_players[nhl_players.columns[16:]] = nhl_players[nhl_players.columns[16:]].apply(pd.to_numeric)
nhl_players.Year = nhl_players.Year.apply(pd.to_numeric)

### 3.2 Split target values

In [73]:
# Rework to get the points from the last season played of all player, and divide by games played, this is the target value
# And remove the features from this last season (so the lines of this player for his last season)

# Get target value (pts/gp in 2017)
# Sort and get pts/gp
nhl_players.sort_values(['Player', 'Year'], inplace=True)
nhl_players['PTS_expected'] = round(82*(nhl_players['Total Points']/nhl_players['GP']))

# Select the last season played by each player
total_points_df = nhl_players.groupby('Player').tail(1)
total_points_df_sub = total_points_df[['Player', 'PTS_expected']].set_index('Player')
total_points = total_points_df_sub['PTS_expected']

# Remove target value instances
# nhl_players = pd.merge(nhl_players, total_points_df, indicator=True, how='outer').query('_merge=="left_only"').drop('_merge', axis=1)

# Keep only the last 5 seasons of each players, to take into account their current pace and not their entire career pace
# eg: Chara is doing less points at 39 than 25
nhl_players = nhl_players.groupby('Player').tail(5)

### 3.3 Remove unecessary features

In [74]:
# Split features
# Keep in memory the player and year order
nhl_players_features_player = nhl_players['Player']
nhl_players_features_year = nhl_players['Year']

# Drop unecessary columns
nhl_players_features = nhl_players.drop(['Team', 'Position', 'Draft Year', 'Date of Birth', 
                                         'Birth City', 'Birth State/Province', 'Birth Country', 
                                         'Nationality', 'Height (in)', 'Weight (lbs)', 
                                         'Draft Team', 'Draft Round', 'Round Pick', 'IPP', 'Overall Draft Position',
                                         'PIM/60', 'Total Penalties/60', 'Minor/60', 'Major/60', 'Misconduct/60',
                                         'Hits/60', 'Hits Taken/60', 'Shots Blocked/60',
                                         'Goals/60', 'Total Assists/60', 'First Assists/60', 'Second Assists/60',
                                         'Goals_PP', 'Total Assists_PP', 'First Assists_PP', 'Second Assists_PP',
                                         'Rush Attempts/60', 'Rebounds Created/60', 'Penalties Drawn/60', 'Giveaways/60',
                                         'Rush Attempts_PP', 'Rebounds Created_PP', 'PIM_PP', 'Total Penalties_PP', 'Minor_PP', 'Major_PP', 'Misconduct_PP', 'Penalties Drawn_PP',
                                         'Takeaways/60', 'Faceoffs Won/60', 'Faceoffs Lost/60', 'Faceoffs %', 'TOI/GP_y',
                                         'Neu. Zone Starts/60', 'Def. Zone Starts/60', 
                                         'On The Fly Starts/60', 'Neu. Zone Faceoffs/60', 'Def. Zone Faceoffs/60',
                                         'Giveaways_PP', 'Takeaways_PP', 'Hits_PP', 'Hits Taken_PP', 'Shots Blocked_PP', 
                                         'Faceoffs Won_PP', 'Faceoffs Lost_PP', 'Faceoffs %_PP',
                                         'CA/60', 'FA/60', 'SA/60', 'GA/60', 'SCA/60', 'HDCA/60', 'HDGA/60', 'MDCA/60', 'MDGA/60', 'LDCA/60', 'LDGA/60', 'SH%', 'On-Ice SV%',
                                         'HDCF/60', 'HDGF/60', 'MDCF/60', 'MDGF/60', 'LDCF/60', 'LDGF/60',
                                         'HDCF%', 'HDGF%', 'MDCF%', 'MDGF%', 'LDCF%', 'LDGF%'], axis=1)

# Sort Values
nhl_players_features = nhl_players_features.fillna(0)
nhl_players_features.sort_values(['Player', 'Year'], inplace=True)

### 3.4 Feature engineering

In [75]:
def normalizeFeature(feature):
    #Normalized Data
    return (feature-min(feature))/(max(feature)-min(feature))

def getPlayerScore(player):
    name = player.name
    return normalizeFeature(features_filtered_direct['CF%__mean'])[name] + normalizeFeature(features_filtered_direct['FF%__mean'])[name] + \
            normalizeFeature(features_filtered_direct['GF%__mean'])[name] + normalizeFeature(features_filtered_direct['GP__mean'])[name] + \
            normalizeFeature(features_filtered_direct['Off. Zone Faceoff %__mean'])[name] + normalizeFeature(features_filtered_direct['Off. Zone Start %__mean'])[name] + \
            normalizeFeature(features_filtered_direct['On-Ice SH%__mean'])[name] - normalizeFeature(features_filtered_direct['PDO__mean'])[name] + \
            normalizeFeature(features_filtered_direct['PTS_expected__mean'])[name] + normalizeFeature(features_filtered_direct['SCF%__mean'])[name] + \
            normalizeFeature(features_filtered_direct['SF%__mean'])[name] + \
            normalizeFeature(features_filtered_direct['Shots/60__mean'])[name] + normalizeFeature(features_filtered_direct['TOI__mean'])[name] + \
            normalizeFeature(features_filtered_direct['TOI_PP__mean'])[name] + normalizeFeature(features_filtered_direct['iSCF/60__mean'])[name]

In [77]:
# Feature Extractor
print('Extract features...')
from tsfresh.feature_extraction import *
features_filtered_direct = tsfresh.extract_features(nhl_players_features, 
                                                    column_id='Player', 
                                                    column_sort='Year',
                                                    default_fc_parameters=tsfresh.feature_extraction.settings.MinimalFCParameters())
 
# Convert NaN and inf
print('Impute NaN and Inf, remove zeros columns...')
features_filtered_direct = tsfresh.utilities.dataframe_functions.impute(features_filtered_direct)

# Remove zeros columns
features_filtered_direct = features_filtered_direct.loc[:, (features_filtered_direct != 0).any(axis=0)]

# Remove columns with same values
cols = list(features_filtered_direct)
nunique = features_filtered_direct.apply(pd.Series.nunique)
cols_to_drop = nunique[nunique == 1].index
features_filtered_direct = features_filtered_direct.drop(cols_to_drop, axis=1)
print('Done.')

# Select relevant features
features_filtered_direct = features_filtered_direct.drop(list(filter(re.compile(".*_length").match, features_filtered_direct.columns)), axis=1)
features_filtered_direct = features_filtered_direct.drop(list(filter(re.compile(".*_sum_values").match, features_filtered_direct.columns)), axis=1)
features_filtered_direct = features_filtered_direct.drop(list(filter(re.compile(".*_standard_deviation").match, features_filtered_direct.columns)), axis=1)
features_filtered_direct = features_filtered_direct.drop(list(filter(re.compile("Age*").match, features_filtered_direct.columns)), axis=1)
# tsfresh.select_features(features_filtered_direct, total_points[:8], ml_task='regression')

Extract features...




Feature Extraction:   0%|          | 0/20 [00:00<?, ?it/s]

Feature Extraction:   5%|▌         | 1/20 [00:00<00:15,  1.23it/s]

Feature Extraction:  10%|█         | 2/20 [00:01<00:11,  1.56it/s]

Feature Extraction:  15%|█▌        | 3/20 [00:01<00:11,  1.50it/s]

Feature Extraction:  20%|██        | 4/20 [00:02<00:08,  1.88it/s]

Feature Extraction:  25%|██▌       | 5/20 [00:02<00:06,  2.28it/s]

Feature Extraction:  30%|███       | 6/20 [00:02<00:05,  2.66it/s]

Feature Extraction:  35%|███▌      | 7/20 [00:02<00:04,  3.08it/s]

Feature Extraction:  40%|████      | 8/20 [00:02<00:03,  3.43it/s]

Feature Extraction:  45%|████▌     | 9/20 [00:03<00:03,  3.56it/s]

Feature Extraction:  50%|█████     | 10/20 [00:03<00:02,  3.87it/s]

Feature Extraction:  55%|█████▌    | 11/20 [00:03<00:02,  3.58it/s]

Feature Extraction:  60%|██████    | 12/20 [00:03<00:02,  3.96it/s]

Feature Extraction:  65%|██████▌   | 13/20 [00:04<00:03,  2.22it/s]

Feature Extraction:  70%|███████   | 14/20 [00:05<

Impute NaN and Inf, remove zeros columns...
Done.


In [78]:
# Add current age, draft position
features_filtered_direct['Age'] = nhl_players.groupby('Player').tail(1).Age.values
features_filtered_direct['Overall Draft Position'] = nhl_players.groupby('Player').tail(1)['Overall Draft Position'].values

# Add stats from last 5 years
year_by_year = nhl_players_features.groupby('Player').tail(5).pivot_table(index='Player', columns='Year', values=nhl_players_features.columns)
year_by_year.columns = ['_'.join(str(i) for i in col) for col in year_by_year.columns.values]

# TODO: make sur we keep only 5 years
for yr in ['2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015']:
    year_by_year = year_by_year.drop(list(filter(re.compile(f".*_{yr}").match, year_by_year.columns)), axis=1)
features_filtered_direct = pd.concat([features_filtered_direct, year_by_year], axis=1)
features_filtered_direct['Position'] = nhl_players.groupby('Player').tail(1).Position.values
features_filtered_direct['Team'] = nhl_players.groupby('Player').tail(1).Team.values

In [79]:
# Get player score
features_filtered_direct['Score'] = features_filtered_direct.apply(lambda player: getPlayerScore(player), axis=1)

# Saving feature names for later use
feature_list = list(features_filtered_direct.columns)
features_filtered_direct.sort_values('Score', ascending=False).head(10)

,CF%__maximum,CF%__mean,CF%__median,CF%__minimum,CF%__variance,CF/60__maximum,CF/60__mean,CF/60__median,CF/60__minimum,CF/60__variance,FF%__maximum,FF%__mean,FF%__median,FF%__minimum,FF%__variance,FF/60__maximum,FF/60__mean,FF/60__median,FF/60__minimum,FF/60__variance,GF%__maximum,GF%__mean,GF%__median,GF%__minimum,GF%__variance,GF/60__maximum,GF/60__mean,GF/60__median,GF/60__minimum,GF/60__variance,GP__maximum,GP__mean,GP__median,GP__minimum,GP__variance,IPP_PP__maximum,IPP_PP__mean,IPP_PP__median,IPP_PP__minimum,IPP_PP__variance,Off. Zone Faceoff %__maximum,Off. Zone Faceoff %__mean,Off. Zone Faceoff %__median,Off. Zone Faceoff %__minimum,Off. Zone Faceoff %__variance,Off. Zone Faceoffs/60__maximum,Off. Zone Faceoffs/60__mean,Off. Zone Faceoffs/60__median,Off. Zone Faceoffs/60__minimum,Off. Zone Faceoffs/60__variance,Off. Zone Start %__maximum,Off. Zone Start %__mean,Off. Zone Start %__median,Off. Zone Start %__minimum,Off. Zone Start %__variance,Off. Zone Starts/60__maximum,Off. Zone Starts/60__mean,Off. Zone Starts/60__median,Off. Zone Starts/60__minimum,Off. Zone Starts/60__variance,On-Ice SH%__maximum,On-Ice SH%__mean,On-Ice SH%__median,On-Ice SH%__minimum,On-Ice SH%__variance,PDO__maximum,PDO__mean,PDO__median,PDO__minimum,PDO__variance,PTS_expected__maximum,PTS_expected__mean,PTS_expected__median,PTS_expected__minimum,PTS_expected__variance,SCF%__maximum,SCF%__mean,SCF%__median,SCF%__minimum,SCF%__variance,SCF/60__maximum,SCF/60__mean,SCF/60__median,SCF/60__minimum,SCF/60__variance,SF%__maximum,SF%__mean,SF%__median,SF%__minimum,SF%__variance,SF/60__maximum,SF/60__mean,SF/60__median,SF/60__minimum,SF/60__variance,SH%_PP__maximum,SH%_PP__mean,SH%_PP__median,SH%_PP__minimum,SH%_PP__variance,Shots/60__maximum,Shots/60__mean,Shots/60__median,Shots/60__minimum,Shots/60__variance,Shots_PP__maximum,Shots_PP__mean,Shots_PP__median,Shots_PP__minimum,Shots_PP__variance,TOI/GP_x__maximum,TOI/GP_x__mean,TOI/GP_x__median,TOI/GP_x__minimum,TOI/GP_x__variance,TOI_PP__maximum,TOI_PP__mean,TOI_PP__median,TOI_PP__minimum,TOI_PP__variance,TOI__maximum,TOI__mean,TOI__median,TOI__minimum,TOI__variance,Total Points/60__maximum,Total Points/60__mean,Total Points/60__median,Total Points/60__minimum,Total Points/60__variance,Total Points/GP__maximum,Total Points/GP__mean,Total Points/GP__median,Total Points/GP__minimum,Total Points/GP__variance,Total Points_PP__maximum,Total Points_PP__mean,Total Points_PP__median,Total Points_PP__minimum,Total Points_PP__variance,Total Points__maximum,Total Points__mean,Total Points__median,Total Points__minimum,Total Points__variance,iCF/60__maximum,iCF/60__mean,iCF/60__median,iCF/60__minimum,iCF/60__variance,iCF_PP__maximum,iCF_PP__mean,iCF_PP__median,iCF_PP__minimum,iCF_PP__variance,iFF/60__maximum,iFF/60__mean,iFF/60__median,iFF/60__minimum,iFF/60__variance,iFF_PP__maximum,iFF_PP__mean,iFF_PP__median,iFF_PP__minimum,iFF_PP__variance,iHDCF/60__maximum,iHDCF/60__mean,iHDCF/60__median,iHDCF/60__minimum,iHDCF/60__variance,iHDCF_PP__maximum,iHDCF_PP__mean,iHDCF_PP__median,iHDCF_PP__minimum,iHDCF_PP__variance,iSCF/60__maximum,iSCF/60__mean,iSCF/60__median,iSCF/60__minimum,iSCF/60__variance,iSCF_PP__maximum,iSCF_PP__mean,iSCF_PP__median,iSCF_PP__minimum,iSCF_PP__variance,ixG/60__maximum,ixG/60__mean,ixG/60__median,ixG/60__minimum,ixG/60__variance,ixG_PP__maximum,ixG_PP__mean,ixG_PP__median,ixG_PP__minimum,ixG_PP__variance,xGA/60__maximum,xGA/60__mean,xGA/60__median,xGA/60__minimum,xGA/60__variance,xGF%__maximum,xGF%__mean,xGF%__median,xGF%__minimum,xGF%__variance,xGF/60__maximum,xGF/60__mean,xGF/60__median,xGF/60__minimum,xGF/60__variance,Age,Overall Draft Position,Age_2016,Age_2017,Age_2018,Age_2019,Age_2020,CF%_2016,CF%_2017,CF%_2018,CF%_2019,CF%_2020,CF/60_2016,CF/60_2017,CF/60_2018,CF/60_2019,CF/60_2020,FF%_2016,FF%_2017,FF%_2018,FF%_2019,FF%_2020,FF/60_2016,FF/60_2017,FF/60_2018,FF/60_2019,FF/60_2020,GF%_2016,GF%_2017,GF%_2018,GF%_2019,GF%_2020,GF/60_2016,GF/60_2017,GF/60_2018,GF/60_2019,GF/60_2020,GP_2016,

### 3.5 Name Standardization

**TODO: Find a better way to deal with wrong names**

In [80]:
features_filtered_direct['Name'] = features_filtered_direct.apply(lambda player: re.sub(r'\W+', '', f'{player.name}').upper(), axis=1)
nhl = features_filtered_direct.copy()
nhl = nhl.set_index('Name')
nhl = nhl.rename({'BURMISTROVALEX': 'BURMISTROVALEXANDER',
                  'BURROWSALEX': 'BURROWSALEXANDRE',
                  'CAMMALLERIMIKE': 'CAMMALLERIMICHAEL',
                  'DEANGELOTONY': 'DEANGELOANTHONY',
                  'DIDOMENICOCHRIS': 'DIDOMENICOCHRISTOPHER',
                  'EDLERALEX': 'EDLERALEXANDER',
                  'ENSTROMTOBY': 'ENSTROMTOBIAS',
                  'FERLANDMICHEAL': 'FERLANDMICHAEL',
                  'HINOSTROZAVINNIE': 'HINOSTROZAVINCENT',
                  'KULEMINNIKOLAY': 'KULEMINNIKOLAI',
                  'MARCHESSAULTJONATHAN': 'AUDYMARCHESSAULTJONATHAN',
                  'MATHESONMIKE': 'MATHESONMICHAEL',
                  'MORRISSEYJOSH': 'MORRISSEYJOSHUA',
                  'PETROVICALEX': 'PETROVICALEXANDER',
                  'RADULOVALEX': 'RADULOVALEXANDER',
                  'WENNBERGALEX': 'WENNBERGALEXANDER'})
nhl.head(10)

,CF%__maximum,CF%__mean,CF%__median,CF%__minimum,CF%__variance,CF/60__maximum,CF/60__mean,CF/60__median,CF/60__minimum,CF/60__variance,FF%__maximum,FF%__mean,FF%__median,FF%__minimum,FF%__variance,FF/60__maximum,FF/60__mean,FF/60__median,FF/60__minimum,FF/60__variance,GF%__maximum,GF%__mean,GF%__median,GF%__minimum,GF%__variance,GF/60__maximum,GF/60__mean,GF/60__median,GF/60__minimum,GF/60__variance,GP__maximum,GP__mean,GP__median,GP__minimum,GP__variance,IPP_PP__maximum,IPP_PP__mean,IPP_PP__median,IPP_PP__minimum,IPP_PP__variance,Off. Zone Faceoff %__maximum,Off. Zone Faceoff %__mean,Off. Zone Faceoff %__median,Off. Zone Faceoff %__minimum,Off. Zone Faceoff %__variance,Off. Zone Faceoffs/60__maximum,Off. Zone Faceoffs/60__mean,Off. Zone Faceoffs/60__median,Off. Zone Faceoffs/60__minimum,Off. Zone Faceoffs/60__variance,Off. Zone Start %__maximum,Off. Zone Start %__mean,Off. Zone Start %__median,Off. Zone Start %__minimum,Off. Zone Start %__variance,Off. Zone Starts/60__maximum,Off. Zone Starts/60__mean,Off. Zone Starts/60__median,Off. Zone Starts/60__minimum,Off. Zone Starts/60__variance,On-Ice SH%__maximum,On-Ice SH%__mean,On-Ice SH%__median,On-Ice SH%__minimum,On-Ice SH%__variance,PDO__maximum,PDO__mean,PDO__median,PDO__minimum,PDO__variance,PTS_expected__maximum,PTS_expected__mean,PTS_expected__median,PTS_expected__minimum,PTS_expected__variance,SCF%__maximum,SCF%__mean,SCF%__median,SCF%__minimum,SCF%__variance,SCF/60__maximum,SCF/60__mean,SCF/60__median,SCF/60__minimum,SCF/60__variance,SF%__maximum,SF%__mean,SF%__median,SF%__minimum,SF%__variance,SF/60__maximum,SF/60__mean,SF/60__median,SF/60__minimum,SF/60__variance,SH%_PP__maximum,SH%_PP__mean,SH%_PP__median,SH%_PP__minimum,SH%_PP__variance,Shots/60__maximum,Shots/60__mean,Shots/60__median,Shots/60__minimum,Shots/60__variance,Shots_PP__maximum,Shots_PP__mean,Shots_PP__median,Shots_PP__minimum,Shots_PP__variance,TOI/GP_x__maximum,TOI/GP_x__mean,TOI/GP_x__median,TOI/GP_x__minimum,TOI/GP_x__variance,TOI_PP__maximum,TOI_PP__mean,TOI_PP__median,TOI_PP__minimum,TOI_PP__variance,TOI__maximum,TOI__mean,TOI__median,TOI__minimum,TOI__variance,Total Points/60__maximum,Total Points/60__mean,Total Points/60__median,Total Points/60__minimum,Total Points/60__variance,Total Points/GP__maximum,Total Points/GP__mean,Total Points/GP__median,Total Points/GP__minimum,Total Points/GP__variance,Total Points_PP__maximum,Total Points_PP__mean,Total Points_PP__median,Total Points_PP__minimum,Total Points_PP__variance,Total Points__maximum,Total Points__mean,Total Points__median,Total Points__minimum,Total Points__variance,iCF/60__maximum,iCF/60__mean,iCF/60__median,iCF/60__minimum,iCF/60__variance,iCF_PP__maximum,iCF_PP__mean,iCF_PP__median,iCF_PP__minimum,iCF_PP__variance,iFF/60__maximum,iFF/60__mean,iFF/60__median,iFF/60__minimum,iFF/60__variance,iFF_PP__maximum,iFF_PP__mean,iFF_PP__median,iFF_PP__minimum,iFF_PP__variance,iHDCF/60__maximum,iHDCF/60__mean,iHDCF/60__median,iHDCF/60__minimum,iHDCF/60__variance,iHDCF_PP__maximum,iHDCF_PP__mean,iHDCF_PP__median,iHDCF_PP__minimum,iHDCF_PP__variance,iSCF/60__maximum,iSCF/60__mean,iSCF/60__median,iSCF/60__minimum,iSCF/60__variance,iSCF_PP__maximum,iSCF_PP__mean,iSCF_PP__median,iSCF_PP__minimum,iSCF_PP__variance,ixG/60__maximum,ixG/60__mean,ixG/60__median,ixG/60__minimum,ixG/60__variance,ixG_PP__maximum,ixG_PP__mean,ixG_PP__median,ixG_PP__minimum,ixG_PP__variance,xGA/60__maximum,xGA/60__mean,xGA/60__median,xGA/60__minimum,xGA/60__variance,xGF%__maximum,xGF%__mean,xGF%__median,xGF%__minimum,xGF%__variance,xGF/60__maximum,xGF/60__mean,xGF/60__median,xGF/60__minimum,xGF/60__variance,Age,Overall Draft Position,Age_2016,Age_2017,Age_2018,Age_2019,Age_2020,CF%_2016,CF%_2017,CF%_2018,CF%_2019,CF%_2020,CF/60_2016,CF/60_2017,CF/60_2018,CF/60_2019,CF/60_2020,FF%_2016,FF%_2017,FF%_2018,FF%_2019,FF%_2020,FF/60_2016,FF/60_2017,FF/60_2018,FF/60_2019,FF/60_2020,GF%_2016,GF%_2017,GF%_2018,GF%_2019,GF%_2020,GF/60_2016,GF/60_2017,GF/60_2018,GF/60_2019,GF/60_2020,GP_2016,

## 4. Contracts
---

1. Export the most recent pool spreadsheet and save it to `../data/Pool-Att-1819.csv`, for example, for forwards of the NHL season 2018-2019. It is necessary to get contract ends to avoid rewriting them.

**TODO: Use capfriendly contract tables?**

### 4.1 Forwards

In [15]:
nhl_att_contracts = pd.read_csv('../data/nhl_att_1920_v2.csv', sep=';', header=0, encoding = "utf-8")
# TODO: Cap Hit_18 will be different probably
# nhl_att_contracts = nhl_att_contracts.rename(columns={'Unnamed: 95': "Cap Hit_18", 'Unnamed: 96': "END", "Unnamed: 0": "Name"})
# nhl_att_contracts['Name'] = nhl_att_contracts.apply(lambda player: re.sub(r'\W+', '', f'{player["Unnamed: 3"]}{player["Unnamed: 2"]}').upper(), axis=1)
nhl_att_contracts.set_index('Name', inplace=True)
nhl_att_contracts.rename({'BURMISTROVALEX': 'BURMISTROVALEXANDER',
                          'BURROWSALEX': 'BURROWSALEXANDRE',
                          'CAMMALLERIMIKE': 'CAMMALLERIMICHAEL',
                          'DEANGELOTONY': 'DEANGELOANTHONY',
                          'DIDOMENICOCHRIS': 'DIDOMENICOCHRISTOPHER',
                          'EDLERALEX': 'EDLERALEXANDER',
                          'ENSTROMTOBY': 'ENSTROMTOBIAS',
                          'FERLANDMICHEAL': 'FERLANDMICHAEL',
                          'HINOSTROZAVINNIE': 'HINOSTROZAVINCENT',
                          'KULEMINNIKOLAY': 'KULEMINNIKOLAI',
                          'MARCHESSAULTJONATHAN': 'AUDYMARCHESSAULTJONATHAN',
                          'MATHESONMIKE': 'MATHESONMICHAEL',
                          'MORRISSEYJOSH': 'MORRISSEYJOSHUA',
                          'PETROVICALEX': 'PETROVICALEXANDER',
                          'RADULOVALEX': 'RADULOVALEXANDER',
                          'WENNBERGALEX': 'WENNBERGALEXANDER'}, inplace = True)
nhl_att_contracts.head()

,Last Name,First Name,Name.1,x,TM,Age,POS,GP_18,GP_mean,GP_var,PTS_proj,PTS_18,PTS_mean,PTS_var,Preds,LSTM_point,LSTM_full,HMM_mean,HMM_min,HMM_max,HMM_var,PTS/GP_18,PTS/GP_mean,PTS/GP_var,$/PTS_exp,SH%_18,SH%_mean,SH%_var,TOI/GP_18,PPTOI/GP_18,OZ%_18,iSCF/60_18,PDO_18,Cap_Hit,End,Score
Name,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
ADAMBURISH,NaN,NaN,nan nan,NaN,NaN,31,R,NaN,20.0,0.000000,NaN,NaN,3.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.150000,0.000000,NaN,NaN,4.690000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.214704
ADAMCRACKNELL,Cracknell,Adam,Adam Cracknell,NaN,NaN,31,R,NaN,46.0,468.666667,NaN,NaN,8.000000,38.000000,NaN,17.936260,16.406738,17.008891,4.878294,32.332530,15.922646,NaN,0.130156,0.008798,NaN,NaN,4.983333,7.159756,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.123663
ADAMERNE,Erne,Adam,Adam Erne,NaN,DET,23,L,65.0,38.0,366.000000,NaN,19.0,8.333333,56.888889,NaN,22.540695,21.351677,19.009138,6.987096,35.229647,27.387945,0.292308,0.179376,0.006415,NaN,10.79,7.400000,9.136067,10.548205,0.498205,48.04,7.26,1.044,1050000.0,NaN,5.043286
ADAMGAUDETTE,Gaudette,Adam,Adam Gaudette,NaN,NaN,22,C,56.0,30.5,650.250000,NaN,11.0,5.500000,30.250000,NaN,16.619806,16.658838,16.150230,5.782005,27.044848,11.967696,0.196429,0.098214,0.009646,NaN,8.03,7.350000,0.462400,10.949107,1.368452,77.60,6.26,0.999,"916,666",20.0,5.059960
ADAMHENRIQUE,Henrique,Adam,Adam Henrique,NaN,ANA,28,C,82.0,80.0,6.800000,NaN,42.0,44.400000,15.840000,NaN,46.334499,43.393280,44.339688,24.089158,63.135351,46.314462,0.512195,0.555715,0.002854,NaN,10.59,10.742000,0.760936,16.451220,2.309350,58.06,6.67,1.015,5825000.0,NaN,6.360994


In [16]:
# nhl_att_contracts = pd.read_csv('../data/Pool-Att-1819.csv', header=1, encoding = "utf-8")
# # TODO: Cap Hit_18 will be different probably
# nhl_att_contracts = nhl_att_contracts.rename(columns={'Unnamed: 95': "Cap Hit_18", 'Unnamed: 96': "END", "Unnamed: 0": "Name"})
# nhl_att_contracts['Name'] = nhl_att_contracts.apply(lambda player: re.sub(r'\W+', '', f'{player["Unnamed: 3"]}{player["Unnamed: 2"]}').upper(), axis=1)
# nhl_att_contracts.set_index('Name', inplace=True)
# nhl_att_contracts.rename({'BURMISTROVALEX': 'BURMISTROVALEXANDER',
#                           'BURROWSALEX': 'BURROWSALEXANDRE',
#                           'CAMMALLERIMIKE': 'CAMMALLERIMICHAEL',
#                           'DEANGELOTONY': 'DEANGELOANTHONY',
#                           'DIDOMENICOCHRIS': 'DIDOMENICOCHRISTOPHER',
#                           'EDLERALEX': 'EDLERALEXANDER',
#                           'ENSTROMTOBY': 'ENSTROMTOBIAS',
#                           'FERLANDMICHEAL': 'FERLANDMICHAEL',
#                           'HINOSTROZAVINNIE': 'HINOSTROZAVINCENT',
#                           'KULEMINNIKOLAY': 'KULEMINNIKOLAI',
#                           'MARCHESSAULTJONATHAN': 'AUDYMARCHESSAULTJONATHAN',
#                           'MATHESONMIKE': 'MATHESONMICHAEL',
#                           'MORRISSEYJOSH': 'MORRISSEYJOSHUA',
#                           'PETROVICALEX': 'PETROVICALEXANDER',
#                           'RADULOVALEX': 'RADULOVALEXANDER',
#                           'WENNBERGALEX': 'WENNBERGALEXANDER'}, inplace = True)
# nhl_att_contracts.head()

### 4.2 Defensemen

In [17]:
nhl_def_contracts = pd.read_csv('../data/nhl_def_1920_v2.csv', sep=';', header=0, encoding = "utf-8")
# TODO: Cap Hit_18 will be different probably
# nhl_def_contracts = nhl_def_contracts.rename(columns={'Unnamed: 95': "Cap Hit_18", 'Unnamed: 96': "END", "Unnamed: 0": "Name"})
# nhl_def_contracts['Name'] = nhl_def_contracts.apply(lambda player: re.sub(r'\W+', '', f'{player["Unnamed: 3"]}{player["Unnamed: 2"]}').upper(), axis=1)
nhl_def_contracts.set_index('Name', inplace=True)
nhl_def_contracts.rename({'BURMISTROVALEX': 'BURMISTROVALEXANDER',
                          'BURROWSALEX': 'BURROWSALEXANDRE',
                          'CAMMALLERIMIKE': 'CAMMALLERIMICHAEL',
                          'DEANGELOTONY': 'DEANGELOANTHONY',
                          'DIDOMENICOCHRIS': 'DIDOMENICOCHRISTOPHER',
                          'EDLERALEX': 'EDLERALEXANDER',
                          'ENSTROMTOBY': 'ENSTROMTOBIAS',
                          'FERLANDMICHEAL': 'FERLANDMICHAEL',
                          'HINOSTROZAVINNIE': 'HINOSTROZAVINCENT',
                          'KULEMINNIKOLAY': 'KULEMINNIKOLAI',
                          'MARCHESSAULTJONATHAN': 'AUDYMARCHESSAULTJONATHAN',
                          'MATHESONMIKE': 'MATHESONMICHAEL',
                          'MORRISSEYJOSH': 'MORRISSEYJOSHUA',
                          'PETROVICALEX': 'PETROVICALEXANDER',
                          'RADULOVALEX': 'RADULOVALEXANDER',
                          'WENNBERGALEX': 'WENNBERGALEXANDER'}, inplace = True)
nhl_def_contracts.head()

,Last Name,First Name,Name.1,x,TM,Age,POS,GP_18,GP_mean,GP_var,PTS_proj,PTS_18,PTS_mean,PTS_var,Preds,LSTM_point,LSTM_full,HMM_mean,HMM_min,HMM_max,HMM_var,PTS/GP_18,PTS/GP_mean,PTS/GP_var,$/PTS_exp,SH%_18,SH%_mean,SH%_var,TOI/GP_18,PPTOI/GP_18,OZ%_18,iSCF/60_18,PDO_18,Cap_Hit,End,Score
Name,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
AARONEKBLAD,Ekblad,Aaron,Aaron Ekblad,NaN,FLA,22,D,82.0,78.2,28.16,NaN,37.0,33.8,48.16,NaN,40.895012,42.660915,34.110372,15.923750,52.954779,47.666184,0.451220,0.427885,0.004507,NaN,10.41,9.200,2.780920,23.644106,1.692683,40.09,2.82,0.985,7500000.0,25.0,6.771978
AARONNESS,Ness,Aaron,Aaron Ness,NaN,NaN,25,D,NaN,8.0,0.00,NaN,NaN,1.0,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.125000,0.000000,NaN,NaN,13.330,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.704962
ADAMCLENDENING,Clendening,Adam,Adam Clendening,NaN,CBJ,26,D,4.0,18.0,132.80,NaN,0.0,4.8,14.96,NaN,24.639565,23.233786,26.870903,13.966267,38.784125,23.487491,0.000000,0.237339,0.019761,NaN,10.53,9.558,0.503976,10.654167,0.287500,60.00,1.41,1.043,700000.0,NaN,4.323720
ADAMLARSSON,Larsson,Adam,Adam Larsson,NaN,EDM,26,D,82.0,74.0,74.80,NaN,20.0,18.4,13.04,NaN,20.192671,20.546745,19.588431,9.164275,29.984558,13.495665,0.243902,0.250754,0.003312,NaN,6.63,8.410,1.336200,21.623171,0.077236,29.91,1.79,0.957,4166666.0,21.0,4.567965
ADAMMCQUAID,McQuaid,Adam,Adam McQuaid,NaN,NaN,32,D,50.0,58.4,177.04,NaN,6.0,6.8,6.16,NaN,11.044761,10.963360,10.219258,2.984972,20.150964,8.828834,0.120000,0.112936,0.000514,NaN,9.97,7.718,1.485656,15.777333,0.011333,35.83,0.99,1.012,NaN,NaN,4.176486


In [18]:
# nhl_def_contracts = pd.read_csv('../data/nhl_def_1920_v2.csv', header=1, encoding = "utf-8")
# # TODO: Cap Hit_18 will be different probably
# nhl_def_contracts = nhl_def_contracts.rename(columns={'Unnamed: 95': "Cap Hit_18", 'Unnamed: 96': "END", "Unnamed: 0": "Name"})
# nhl_def_contracts['Name'] = nhl_def_contracts.apply(lambda player: re.sub(r'\W+', '', f'{player["Unnamed: 3"]}{player["Unnamed: 2"]}').upper(), axis=1)
# nhl_def_contracts.set_index('Name', inplace=True)
# nhl_def_contracts.rename({'BURMISTROVALEX': 'BURMISTROVALEXANDER',
#            'BURROWSALEX': 'BURROWSALEXANDRE',
#            'CAMMALLERIMIKE': 'CAMMALLERIMICHAEL',
#            'DEANGELOTONY': 'DEANGELOANTHONY',
#            'DIDOMENICOCHRIS': 'DIDOMENICOCHRISTOPHER',
#            'EDLERALEX': 'EDLERALEXANDER',
#            'ENSTROMTOBY': 'ENSTROMTOBIAS',
#            'FERLANDMICHEAL': 'FERLANDMICHAEL',
#            'HINOSTROZAVINNIE': 'HINOSTROZAVINCENT',
#            'KULEMINNIKOLAY': 'KULEMINNIKOLAI',
#            'MARCHESSAULTJONATHAN': 'AUDYMARCHESSAULTJONATHAN',
#            'MATHESONMIKE': 'MATHESONMICHAEL',
#            'MORRISSEYJOSH': 'MORRISSEYJOSHUA',
#            'PETROVICALEX': 'PETROVICALEXANDER',
#            'RADULOVALEX': 'RADULOVALEXANDER',
#            'WENNBERGALEX': 'WENNBERGALEXANDER'}, inplace = True)
# nhl_def_contracts.head()

### 4.3 Merge All

In [19]:
nhl_contracts = pd.concat([nhl_att_contracts, nhl_def_contracts], sort=True)
# TODO: Cap Hit_18 will be different probably
nhl = nhl.join(nhl_contracts[['First Name', 'Last Name', 'End', 'Cap_Hit']])
nhl = nhl.rename({'Cap_Hit': "Cap_Hit_19"}, axis=1)

In [20]:
# nhl_contracts = pd.concat([nhl_att_contracts, nhl_def_contracts], sort=True)
# # TODO: Cap Hit_18 will be different probably
# nhl = nhl.join(nhl_contracts[['Unnamed: 2', 'Unnamed: 3', 'END', 'Cap Hit_18']])
# nhl = nhl.rename({'Unnamed: 2': "Last Name", 'Unnamed: 3': 'First Name'}, axis=1)

## 5. Salary Correction
---

1. Update with salaries from 2019-2020 season from https://www.hockey-reference.com/friv/current_nhl_salaries.cgi

2. Save table under `../data/Salaries-2019-2020.csv` for NHL season 2019-2020 for example.

**TODO: Use capfriendly salary tables?**

In [21]:
# Salaries of 2019-2020 season
nhl_salaries = pd.read_csv('../data/Salaries-2019-2020.csv', header=0, sep=';', encoding = "utf-8")
nhl_salaries.columns = ['Player', 'Team', 'Salary', 'Cap_Hit']
nhl_salaries['Name'] = nhl_salaries.Player.apply(lambda player: re.sub(r'\W+', '', f'{player}').upper())
nhl_salaries.set_index('Name', inplace=True)
nhl_salaries.rename({'BURMISTROVALEX': 'BURMISTROVALEXANDER',
                     'BURROWSALEX': 'BURROWSALEXANDRE',
                     'CAMMALLERIMIKE': 'CAMMALLERIMICHAEL',
                     'DEANGELOTONY': 'DEANGELOANTHONY',
                     'DIDOMENICOCHRIS': 'DIDOMENICOCHRISTOPHER',
                     'EDLERALEX': 'EDLERALEXANDER',
                     'ENSTROMTOBY': 'ENSTROMTOBIAS',
                     'FERLANDMICHEAL': 'FERLANDMICHAEL',
                     'HINOSTROZAVINNIE': 'HINOSTROZAVINCENT',
                     'KULEMINNIKOLAY': 'KULEMINNIKOLAI',
                     'MARCHESSAULTJONATHAN': 'AUDYMARCHESSAULTJONATHAN',
                     'MATHESONMIKE': 'MATHESONMICHAEL',
                     'MORRISSEYJOSH': 'MORRISSEYJOSHUA',
                     'PETROVICALEX': 'PETROVICALEXANDER',
                     'RADULOVALEX': 'RADULOVALEXANDER',
                     'WENNBERGALEX': 'WENNBERGALEXANDER'}, inplace = True)
nhl_salaries.head()

,Player,Team,Salary,Cap_Hit
Name,,,,
MITCHMARNER,Mitch Marner,TOR,"16,000,000","10,893,000"
JOHNTAVARES,John Tavares,TOR,"15,900,000","11,000,000"
AUSTONMATTHEWS,Auston Matthews,TOR,"15,900,000","11,634,000"
CONNORMCDAVID,Connor McDavid,EDM,"15,000,000","12,500,000"
CAREYPRICE,Carey Price,MTL,"15,000,000","10,500,000"


In [22]:
# Reverse index of df and join the salaries
nhl = nhl.drop('Team', axis=1)
nhl = nhl.join(nhl_salaries[['Team', 'Cap_Hit']])

# Convert to float
nhl['Cap_Hit'] = nhl['Cap_Hit'].replace(',', '', regex=True).astype(float)

# If contract ended last year, put ending year and cap hit from last year to NaN and add manually
# TODO: END year will be different and Cap Hit_18
nhl.loc[nhl.End <= 20, 'Cap Hit_19'] = np.nan
nhl.loc[nhl.End <= 20, 'End'] = np.nan

# If no match for hockey reference salaries, use contract from last year
# All contracts ending have been assigne NaN, so all remaining contracts are valid, 
# thus, this operation will transfer only good contracts
# TODO: Cap Hit_18 will be different
nhl.loc[~(nhl.Cap_Hit > 0), 'Cap_Hit'] = nhl.loc[~(nhl.Cap_Hit > 0), 'Cap_Hit_19']
nhl = nhl.drop('Cap_Hit_19', axis=1)
nhl.head()

,CF%__maximum,CF%__mean,CF%__median,CF%__minimum,CF%__variance,CF/60__maximum,CF/60__mean,CF/60__median,CF/60__minimum,CF/60__variance,FF%__maximum,FF%__mean,FF%__median,FF%__minimum,FF%__variance,FF/60__maximum,FF/60__mean,FF/60__median,FF/60__minimum,FF/60__variance,GF%__maximum,GF%__mean,GF%__median,GF%__minimum,GF%__variance,GF/60__maximum,GF/60__mean,GF/60__median,GF/60__minimum,GF/60__variance,GP__maximum,GP__mean,GP__median,GP__minimum,GP__variance,IPP_PP__maximum,IPP_PP__mean,IPP_PP__median,IPP_PP__minimum,IPP_PP__variance,Off. Zone Faceoff %__maximum,Off. Zone Faceoff %__mean,Off. Zone Faceoff %__median,Off. Zone Faceoff %__minimum,Off. Zone Faceoff %__variance,Off. Zone Faceoffs/60__maximum,Off. Zone Faceoffs/60__mean,Off. Zone Faceoffs/60__median,Off. Zone Faceoffs/60__minimum,Off. Zone Faceoffs/60__variance,Off. Zone Start %__maximum,Off. Zone Start %__mean,Off. Zone Start %__median,Off. Zone Start %__minimum,Off. Zone Start %__variance,Off. Zone Starts/60__maximum,Off. Zone Starts/60__mean,Off. Zone Starts/60__median,Off. Zone Starts/60__minimum,Off. Zone Starts/60__variance,On-Ice SH%__maximum,On-Ice SH%__mean,On-Ice SH%__median,On-Ice SH%__minimum,On-Ice SH%__variance,PDO__maximum,PDO__mean,PDO__median,PDO__minimum,PDO__variance,PTS_expected__maximum,PTS_expected__mean,PTS_expected__median,PTS_expected__minimum,PTS_expected__variance,SCF%__maximum,SCF%__mean,SCF%__median,SCF%__minimum,SCF%__variance,SCF/60__maximum,SCF/60__mean,SCF/60__median,SCF/60__minimum,SCF/60__variance,SF%__maximum,SF%__mean,SF%__median,SF%__minimum,SF%__variance,SF/60__maximum,SF/60__mean,SF/60__median,SF/60__minimum,SF/60__variance,SH%_PP__maximum,SH%_PP__mean,SH%_PP__median,SH%_PP__minimum,SH%_PP__variance,Shots/60__maximum,Shots/60__mean,Shots/60__median,Shots/60__minimum,Shots/60__variance,Shots_PP__maximum,Shots_PP__mean,Shots_PP__median,Shots_PP__minimum,Shots_PP__variance,TOI/GP_x__maximum,TOI/GP_x__mean,TOI/GP_x__median,TOI/GP_x__minimum,TOI/GP_x__variance,TOI_PP__maximum,TOI_PP__mean,TOI_PP__median,TOI_PP__minimum,TOI_PP__variance,TOI__maximum,TOI__mean,TOI__median,TOI__minimum,TOI__variance,Total Points/60__maximum,Total Points/60__mean,Total Points/60__median,Total Points/60__minimum,Total Points/60__variance,Total Points/GP__maximum,Total Points/GP__mean,Total Points/GP__median,Total Points/GP__minimum,Total Points/GP__variance,Total Points_PP__maximum,Total Points_PP__mean,Total Points_PP__median,Total Points_PP__minimum,Total Points_PP__variance,Total Points__maximum,Total Points__mean,Total Points__median,Total Points__minimum,Total Points__variance,iCF/60__maximum,iCF/60__mean,iCF/60__median,iCF/60__minimum,iCF/60__variance,iCF_PP__maximum,iCF_PP__mean,iCF_PP__median,iCF_PP__minimum,iCF_PP__variance,iFF/60__maximum,iFF/60__mean,iFF/60__median,iFF/60__minimum,iFF/60__variance,iFF_PP__maximum,iFF_PP__mean,iFF_PP__median,iFF_PP__minimum,iFF_PP__variance,iHDCF/60__maximum,iHDCF/60__mean,iHDCF/60__median,iHDCF/60__minimum,iHDCF/60__variance,iHDCF_PP__maximum,iHDCF_PP__mean,iHDCF_PP__median,iHDCF_PP__minimum,iHDCF_PP__variance,iSCF/60__maximum,iSCF/60__mean,iSCF/60__median,iSCF/60__minimum,iSCF/60__variance,iSCF_PP__maximum,iSCF_PP__mean,iSCF_PP__median,iSCF_PP__minimum,iSCF_PP__variance,ixG/60__maximum,ixG/60__mean,ixG/60__median,ixG/60__minimum,ixG/60__variance,ixG_PP__maximum,ixG_PP__mean,ixG_PP__median,ixG_PP__minimum,ixG_PP__variance,xGA/60__maximum,xGA/60__mean,xGA/60__median,xGA/60__minimum,xGA/60__variance,xGF%__maximum,xGF%__mean,xGF%__median,xGF%__minimum,xGF%__variance,xGF/60__maximum,xGF/60__mean,xGF/60__median,xGF/60__minimum,xGF/60__variance,Age,Overall Draft Position,Age_2015,Age_2016,Age_2017,Age_2018,Age_2019,CF%_2015,CF%_2016,CF%_2017,CF%_2018,CF%_2019,CF/60_2015,CF/60_2016,CF/60_2017,CF/60_2018,CF/60_2019,FF%_2015,FF%_2016,FF%_2017,FF%_2018,FF%_2019,FF/60_2015,FF/60_2016,FF/60_2017,FF/60_2018,FF/60_2019,GF%_2015,GF%_2016,GF%_2017,GF%_2018,GF%_2019,GF/60_2015,GF/60_2016,GF/60_2017,GF/60_2018,GF/60_2019,GP_2015,

In [23]:
# Filter to get only desired columns
nhl = nhl.filter(regex=("First Name|Last Name|Age$|Position|Team|^GP|^TOI/GP_x|^Total Points|^PTS_expected|^Total Points/GP|^On-Ice SH%|^TOI_PP|^Total Points_PP|^iSCF|^Off. Zone Start %|^PDO|Score|End|Cap_Hit"))
nhl['Team'] = nhl['Team'].apply(lambda tm: str(tm).split('/')[-1])
# TODO: 'TOI/GP_PP_2018' will be different
nhl['TOI/GP_PP_2019'] = nhl['TOI_PP_2019']/nhl['GP_2019']
nhl = nhl[nhl.index != 'NANNAN']
nhl.head(10)

,GP__maximum,GP__mean,GP__median,GP__minimum,GP__variance,Off. Zone Start %__maximum,Off. Zone Start %__mean,Off. Zone Start %__median,Off. Zone Start %__minimum,Off. Zone Start %__variance,On-Ice SH%__maximum,On-Ice SH%__mean,On-Ice SH%__median,On-Ice SH%__minimum,On-Ice SH%__variance,PDO__maximum,PDO__mean,PDO__median,PDO__minimum,PDO__variance,PTS_expected__maximum,PTS_expected__mean,PTS_expected__median,PTS_expected__minimum,PTS_expected__variance,TOI/GP_x__maximum,TOI/GP_x__mean,TOI/GP_x__median,TOI/GP_x__minimum,TOI/GP_x__variance,TOI_PP__maximum,TOI_PP__mean,TOI_PP__median,TOI_PP__minimum,TOI_PP__variance,Total Points/60__maximum,Total Points/60__mean,Total Points/60__median,Total Points/60__minimum,Total Points/60__variance,Total Points/GP__maximum,Total Points/GP__mean,Total Points/GP__median,Total Points/GP__minimum,Total Points/GP__variance,Total Points_PP__maximum,Total Points_PP__mean,Total Points_PP__median,Total Points_PP__minimum,Total Points_PP__variance,Total Points__maximum,Total Points__mean,Total Points__median,Total Points__minimum,Total Points__variance,iSCF/60__maximum,iSCF/60__mean,iSCF/60__median,iSCF/60__minimum,iSCF/60__variance,iSCF_PP__maximum,iSCF_PP__mean,iSCF_PP__median,iSCF_PP__minimum,iSCF_PP__variance,Age,Overall Draft Position,GP_2015,GP_2016,GP_2017,GP_2018,GP_2019,Off. Zone Start %_2015,Off. Zone Start %_2016,Off. Zone Start %_2017,Off. Zone Start %_2018,Off. Zone Start %_2019,On-Ice SH%_2015,On-Ice SH%_2016,On-Ice SH%_2017,On-Ice SH%_2018,On-Ice SH%_2019,PDO_2015,PDO_2016,PDO_2017,PDO_2018,PDO_2019,PTS_expected_2015,PTS_expected_2016,PTS_expected_2017,PTS_expected_2018,PTS_expected_2019,TOI/GP_x_2015,TOI/GP_x_2016,TOI/GP_x_2017,TOI/GP_x_2018,TOI/GP_x_2019,TOI_PP_2015,TOI_PP_2016,TOI_PP_2017,TOI_PP_2018,TOI_PP_2019,Total Points_2015,Total Points_2016,Total Points_2017,Total Points_2018,Total Points_2019,Total Points/60_2015,Total Points/60_2016,Total Points/60_2017,Total Points/60_2018,Total Points/60_2019,Total Points/GP_2015,Total Points/GP_2016,Total Points/GP_2017,Total Points/GP_2018,Total Points/GP_2019,Total Points_PP_2015,Total Points_PP_2016,Total Points_PP_2017,Total Points_PP_2018,Total Points_PP_2019,iSCF/60_2015,iSCF/60_2016,iSCF/60_2017,iSCF/60_2018,iSCF/60_2019,iSCF_PP_2015,iSCF_PP_2016,iSCF_PP_2017,iSCF_PP_2018,iSCF_PP_2019,Position,Score,First Name,Last Name,End,Team,Cap_Hit,TOI/GP_PP_2019
Name,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
AARONEKBLAD,82.0,75.4,78.0,67.0,43.840000,68.16,50.754000,42.980,40.09,127.286704,10.80,9.668000,10.410,6.24,2.964376,1.019,0.997200,0.990,0.974,0.000329,50.0,37.400000,38.0,24.0,67.840000,23.644106,22.624859,22.953980,21.460049,0.797219,221.533333,164.916667,175.466667,98.316667,1817.438444,1.60,1.216000,1.190,0.86,0.056664,0.611940,0.456446,0.461538,0.294118,0.010126,11.0,8.800000,9.0,6.0,2.960000,41.0,34.400000,37.0,20.0,54.640000,4.56,3.384000,3.05,2.77,0.460504,32.0,19.200000,20.0,5.0,81.360000,23,1,78.0,68.0,82.0,82.0,67.0,60.18,68.16,42.36,40.09,42.98,10.57,6.24,10.32,10.41,10.80,1.019,0.974,1.018,0.985,0.990,38.0,24.0,38.0,37.0,50.0,21.677137,21.460049,23.389024,23.644106,22.953980,221.533333,190.466667,175.466667,138.800000,98.316667,36.0,20.0,38.0,37.0,41.0,1.28,0.86,1.19,1.15,1.60,0.461538,0.294118,0.463415,0.451220,0.611940,9.0,8.0,11.0,10.0,6.0,3.05,4.56,3.72,2.82,2.77,20.0,32.0,24.0,15.0,5.0,D,6.471522,Aaron,Ekblad,25.0,FLA,7.5e+06,1.467413
AARONNESS,24.0,16.0,16.0,8.0,64.000000,81.25,74.645000,74.645,68.04,43.626025,13.33,10.860000,10.860,8.39,6.100900,1.082,1.042000,1.042,1.002,0.001600,10.0,6.500000,6.5,3.0,12.250000,12.534722,12.461111,12.461111,12.387500,0.005419,2.033333,1.091667,1.091667,0.150000,0.886736,1.21,0.705000,0.705,0.20,0.255025,0.125000,0.083333,0.083333,0.041667,0.001736,0.0,0.000000,0.0,0.0,0.000000,1.0,1.000000,1.0,1.0,0.000000,1.82,1.210000,1.21,0.60,0.372100,0.0,0.000000,0.0,0.0,0.000000,29,40,8.0,NaN,NaN,Na

## 6. Model Predictions
---


### 6.1 LSTM

In [24]:
lstm_preds = []
# TODO: Make sure predictions are updated
for player in glob.glob('../LSTM-Neural-Network-for-Time-Series-Prediction/predictions/*.pkl'):
    tmp = pd.read_pickle(player)
    tmp = pd.DataFrame.from_dict(tmp, orient='index')
    lstm_preds.append(tmp)
lstm_preds = pd.concat(lstm_preds)
lstm_preds['Player_Id'] = lstm_preds.apply(lambda x: x.name.split('.cs')[0].split('-')[-1], axis=1)
lstm_preds['LSTM_full'] = lstm_preds.apply(lambda x: np.array(x['full-sequence']).sum(), axis=1)
lstm_preds['LSTM_point'] = lstm_preds.apply(lambda x: np.array(x['point-by-point']).sum(), axis=1)
lstm_preds.index.name = 'name'
lstm_preds.head()

,multiple-sequences,full-sequence,point-by-point,train,test,Player_Id,LSTM_full,LSTM_point
name,,,,,,,,
AARON-DELL-8477180.csv,"[[-0.016900066, -0.016693277, -0.016369687, -0...","[-0.016900066, -0.016693277, -0.016369687, -0....","[-0.016900063, -0.016900063, -0.016900063, -0....","[2016-10-18 00:00:00, 2020-02-25 00:00:00]","[2020-03-11 00:00:00, 2020-03-11 00:00:00]",8477180,-1.296410,-1.385805
AARON-EKBLAD-8477932.csv,"[[0.54700744, 0.6109164, 0.59403276, 0.5983989...","[0.54700744, 0.6109164, 0.59403276, 0.5983989,...","[0.54700744, 0.54700744, 0.54700744, 0.5470074...","[2014-10-09 00:00:00, 2020-03-05 00:00:00]","[2020-03-09 00:00:00, 2020-03-09 00:00:00]",8477932,47.702736,44.854618
AARON-NESS-8474604.csv,"[[0.13768713, 0.13522127, 0.12800954, 0.117788...","[0.13768713, 0.13522127, 0.12800954, 0.1177882...","[0.13768713, 0.13768713, 0.13768713, 0.1376871...","[2013-11-07 00:00:00, 2020-02-22 00:00:00]","[2020-02-25 00:00:00, 2020-02-25 00:00:00]",8474604,9.247370,11.290343
AARON-VOLPATTI-8475619.csv,"[[0.031375676, 0.03074418, 0.029949684, 0.0292...","[0.031375676, 0.03074418, 0.029949684, 0.02925...","[0.03137568, 0.03137568, 0.03137568, 0.0313756...","[2013-10-19 00:00:00, 2015-02-03 00:00:00]","[2015-02-14 00:00:00, 2015-02-14 00:00:00]",8475619,2.372661,2.572806
ADAM-CLENDENING-8476431.csv,"[[0.2944563, 0.30810884, 0.30491444, 0.2907306...","[0.2944563, 0.30810884, 0.30491444, 0.29073068...","[0.2944563, 0.2944563, 0.2944563, 0.2944563, 0...","[2014-11-20 00:00:00, 2018-10-11 00:00:00]","[2018-10-13 00:00:00, 2018-10-13 00:00:00]",8476431,22.819145,24.145414


### 6.2 HMM

In [25]:
# TODO: Make sure predictions are updated (run HMMLearn.ipynb)
hmm_preds = pd.read_pickle(os.path.join(data_folder, 'predictions_hmm_3years_for_2021.pkl'))
hmm_preds.index.name = 'name'
hmm_preds.head()

,nobs,minmax,mean,variance,skewness,kurtosis
name,,,,,,
A.J.-GREER-8478421.csv,500,"(5.709672724683751, 25.534055004993228)",13.1009,7.60412,0.227929,0.522997
AARON-DELL-8477180.csv,500,"(-0.23544148291288167, 4.213466197174426)",0.925224,0.929644,0.854183,0.174074
AARON-EKBLAD-8477932.csv,500,"(18.98625378121845, 58.96552562304788)",40.52,48.4988,0.0443088,-0.160787
AARON-NESS-8474604.csv,500,"(-0.0400711438583026, 12.910477808168023)",5.11348,4.6331,0.482452,0.312154
AARON-PALUSHAJ-8474030.csv,0,0,0,0,0,0


### 6.3 Merge predictions

In [26]:
predictions = lstm_preds.merge(hmm_preds, on='name')
predictions.head()

,multiple-sequences,full-sequence,point-by-point,train,test,Player_Id,LSTM_full,LSTM_point,nobs,minmax,mean,variance,skewness,kurtosis
name,,,,,,,,,,,,,,
AARON-DELL-8477180.csv,"[[-0.016900066, -0.016693277, -0.016369687, -0...","[-0.016900066, -0.016693277, -0.016369687, -0....","[-0.016900063, -0.016900063, -0.016900063, -0....","[2016-10-18 00:00:00, 2020-02-25 00:00:00]","[2020-03-11 00:00:00, 2020-03-11 00:00:00]",8477180,-1.296410,-1.385805,500,"(-0.23544148291288167, 4.213466197174426)",0.925224,0.929644,0.854183,0.174074
AARON-EKBLAD-8477932.csv,"[[0.54700744, 0.6109164, 0.59403276, 0.5983989...","[0.54700744, 0.6109164, 0.59403276, 0.5983989,...","[0.54700744, 0.54700744, 0.54700744, 0.5470074...","[2014-10-09 00:00:00, 2020-03-05 00:00:00]","[2020-03-09 00:00:00, 2020-03-09 00:00:00]",8477932,47.702736,44.854618,500,"(18.98625378121845, 58.96552562304788)",40.52,48.4988,0.0443088,-0.160787
AARON-NESS-8474604.csv,"[[0.13768713, 0.13522127, 0.12800954, 0.117788...","[0.13768713, 0.13522127, 0.12800954, 0.1177882...","[0.13768713, 0.13768713, 0.13768713, 0.1376871...","[2013-11-07 00:00:00, 2020-02-22 00:00:00]","[2020-02-25 00:00:00, 2020-02-25 00:00:00]",8474604,9.247370,11.290343,500,"(-0.0400711438583026, 12.910477808168023)",5.11348,4.6331,0.482452,0.312154
AARON-VOLPATTI-8475619.csv,"[[0.031375676, 0.03074418, 0.029949684, 0.0292...","[0.031375676, 0.03074418, 0.029949684, 0.02925...","[0.03137568, 0.03137568, 0.03137568, 0.0313756...","[2013-10-19 00:00:00, 2015-02-03 00:00:00]","[2015-02-14 00:00:00, 2015-02-14 00:00:00]",8475619,2.372661,2.572806,0,0,0,0,0,0
ADAM-CLENDENING-8476431.csv,"[[0.2944563, 0.30810884, 0.30491444, 0.2907306...","[0.2944563, 0.30810884, 0.30491444, 0.29073068...","[0.2944563, 0.2944563, 0.2944563, 0.2944563, 0...","[2014-11-20 00:00:00, 2018-10-11 00:00:00]","[2018-10-13 00:00:00, 2018-10-13 00:00:00]",8476431,22.819145,24.145414,0,0,0,0,0,0


### 6.4 Merge to stats

In [27]:
nhl['merge_on'] = nhl.apply(lambda player: re.sub(r'\W+', '', f'{player["First Name"]}{player["Last Name"]}').upper(), axis=1)
predictions['merge_on'] = predictions.apply(lambda player: ''.join(player.name.split('-')[:-1]), axis=1)
predictions.replace({'ALEXBURMISTROV': 'ALEXANDERBURMISTROV',
                    'ALEXBURROWS': 'ALEXANDREBURROWS',
                    'MIKECAMMALLERI': 'MICHAELCAMMALLERI',
                    'ANTHONYDEANGELO': 'TONYDEANGELO',
                    'CHRISDIDOMENICO': 'CHRISTOPHERDIDOMENICO',
#                     'ALEXANDEREDLER': 'ALEXEDLER',
#                     'TOBYENSTROM': 'TOBIASENSTROM',
                    'MICHAELFERLAND': 'MICHEALFERLAND',
                    'VINCENTHINNOSTROZA': 'VINNIEHINNOSTROZA',
#                     'NIKOLAYKULEMIN': 'NIKOLAIKULEMIN',
                    'JONATHANAUDYMARCHESSAULT': 'JONATHANMARCHESSAULT',
#                     'MICHAELMATHESON': 'MIKEMATHESON',
                    'JOSHUAMORRISSEY': 'JOSHMORRISSEY',
                    'ALEXANDERPETROVIC': 'ALEXPETROVIC',
#                     'ALEXANDERRADULOV': 'ALEXRADULOV',
#                     'ALEXANDERWENNBERG': 'ALEXWENNBERG',
#                     'ALEXANDERSTEEN': 'ALEXSTEEN',
#                     'ALEXANDERKERFOOT': 'ALEXKERFOOT',
#                     'CHRISTOPHERTANEV': 'CHRISTANEV',
#                     'JTMILLER': 'JMILLER',
#                     'MITCHELLMARNER': 'MITCHMARNER',
#                     'PKSUBBAN': 'PSUBBAN',
                    "RYANO'REILLY": 'RYANOREILLY',
#                     'TJOSHIE': 'TOSHIE'
                    }, 
                    inplace=True)
nhl = nhl.join(predictions.set_index('merge_on'), on='merge_on', lsuffix='_nhl', rsuffix='_preds')
nhl = nhl.drop(['merge_on', 'multiple-sequences', 'full-sequence', 'point-by-point', 'train', 'test', 'nobs'], axis=1)
nhl['min'], nhl['max'] = nhl.minmax.str

/home/cenoi/miniconda3/envs/pool/lib/python3.7/site-packages/pandas/core/internals/managers.py:1937: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  op = lambda x: operator.eq(x, b)


In [28]:
# To fill in excel
nhl['x'] = np.nan
nhl['PTS_proj'] = np.nan
nhl['preds'] = np.nan
nhl['$/PTS_exp'] = np.nan
nhl['Name'] = nhl.apply(lambda player: f'{player["First Name"]} {player["Last Name"]}', axis=1)

## 7. Save Results
---

In [29]:
# TODO: Fix column names (the ones with a year suffix especially, 2018 to 2019, 2019 to 2020, etc)
nhl = nhl[['Last Name', 'First Name', 'Name', 'x', 'Team', 'Age', 'Position', 
           'GP_2019', 'GP__mean', 'GP__variance',
           'PTS_proj', 'Total Points_2019', 'Total Points__mean', 'Total Points__variance', 
            # 'PTS_expected_2018', 'PTS_expected__mean', 'PTS_expected__variance',
           'preds', 'LSTM_point', 'LSTM_full', 'mean', 'min', 'max', 'variance',
           'Total Points/GP_2019', 'Total Points/GP__mean', 'Total Points/GP__variance', 
           '$/PTS_exp',
            # '$/PTS_13', '$/PTS_14', '$/PTS_15', '$/PTS_16', '$/PTS_17', '$/PTS_18', '$/PTS_19',
           'On-Ice SH%_2019', 'On-Ice SH%__mean', 'On-Ice SH%__variance', 
           'TOI/GP_x_2019', #'TOI/GP_x__mean', 'TOI/GP_x__variance', 
           'TOI/GP_PP_2019',
           'Off. Zone Start %_2019', #'Off. Zone Start %__mean', 'Off. Zone Start %__variance', 
           'iSCF/60_2019', #'iSCF/60__mean', 'iSCF/60__variance',
           'PDO_2019', #'PDO__mean', 'PDO__variance',
           'Cap_Hit', 
           'End',
           'Score']]

# TODO: Fix column names (the ones with a year suffix especially, 2018 to 2019, 2019 to 2020, etc)
nhl.columns = ['Last Name', 'First Name', 'Name', 'x', 'TM', 'Age', 'POS', 
               'GP_19', 'GP_mean', 'GP_var',
               'PTS_proj', 'PTS_19', 'PTS_mean', 'PTS_var', 
               'Preds', 'LSTM_point', 'LSTM_full', 'HMM_mean', 'HMM_min', 'HMM_max', 'HMM_var',
               'PTS/GP_19', 'PTS/GP_mean', 'PTS/GP_var', 
               '$/PTS_exp',
               'SH%_19', 'SH%_mean', 'SH%_var', 
               'TOI/GP_19', 
               'PPTOI/GP_19',
               'OZ%_19', 
               'iSCF/60_19', 
               'PDO_19', 
               'Cap_Hit', 
               'End',
               'Score']

nhl.sort_values('Score', ascending=False).head(10)

,Last Name,First Name,Name,x,TM,Age,POS,GP_19,GP_mean,GP_var,PTS_proj,PTS_19,PTS_mean,PTS_var,Preds,LSTM_point,LSTM_full,HMM_mean,HMM_min,HMM_max,HMM_var,PTS/GP_19,PTS/GP_mean,PTS/GP_var,$/PTS_exp,SH%_19,SH%_mean,SH%_var,TOI/GP_19,PPTOI/GP_19,OZ%_19,iSCF/60_19,PDO_19,Cap_Hit,End,Score
Name,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
ALEXOVECHKIN,Ovechkin,Alex,Alex Ovechkin,NaN,WSH,34,L,68.0,78.4,28.24,NaN,66.0,76.0,83.60,NaN,84.131401,79.006073,90.590616,59.984571,116.224439,76.218382,0.970588,0.969197,0.008272,NaN,11.59,12.136,0.589624,20.648775,4.868627,69.77,12.01,0.993,9.53846e+06,21.0,8.967810
NIKITAKUCHEROV,Kucherov,Nikita,Nikita Kucherov,NaN,TBL,26,R,68.0,76.2,24.16,NaN,85.0,92.0,423.20,NaN,81.190643,97.366119,113.273235,80.935038,142.757935,118.360430,1.250000,1.203114,0.051268,NaN,12.87,12.424,1.363784,18.863235,3.513235,75.58,9.59,1.044,9.5e+06,NaN,8.685755
PATRICKKANE,Kane,Patrick,Patrick Kane,NaN,CHI,31,R,70.0,79.4,22.24,NaN,83.0,92.0,168.80,NaN,93.193115,112.154762,100.171210,73.150001,125.779858,93.926527,1.185714,1.159937,0.023480,NaN,10.80,11.452,2.273176,21.340714,3.700000,77.62,10.64,1.027,1.05e+07,23.0,8.610506
ARTEMIPANARIN,Panarin,Artemi,Artemi Panarin,NaN,NYR,28,"C, L",69.0,78.2,22.16,NaN,95.0,82.8,59.36,NaN,103.424553,114.863617,98.692289,68.651325,127.689305,93.291927,1.376812,1.068633,0.028459,NaN,13.93,11.854,2.438944,20.603865,3.659903,66.30,8.61,1.057,1.16429e+07,NaN,8.478510
SIDNEYCROSBY,Crosby,Sidney,Sidney Crosby,NaN,PIT,32,C,41.0,71.4,236.24,NaN,46.0,81.4,341.44,NaN,104.459442,106.342499,97.395272,61.942555,129.878223,129.725384,1.121951,1.139295,0.005670,NaN,10.02,11.150,1.119920,20.266260,3.805285,75.80,7.08,0.985,8.7e+06,25.0,8.385461
EVGENIMALKIN,Malkin,Evgeni,Evgeni Malkin,NaN,PIT,33,C,55.0,64.0,69.20,NaN,74.0,74.4,175.44,NaN,94.329292,94.871185,101.332370,70.918844,135.806800,90.349758,1.345455,1.161170,0.015948,NaN,11.75,11.500,0.872600,19.521212,3.616061,78.14,9.39,1.006,9.5e+06,22.0,8.354665
NATHANMACKINNON,MacKinnon,Nathan,Nathan MacKinnon,NaN,COL,24,C,69.0,75.8,28.16,NaN,92.0,78.2,462.56,NaN,101.048759,104.437973,105.602758,78.614302,133.393114,83.336620,1.333333,1.038863,0.089187,NaN,12.36,11.288,3.213416,21.222464,4.111836,70.48,11.39,1.020,6.3e+06,23.0,8.341883
PHILKESSEL,Kessel,Phil,Phil Kessel,NaN,ARI,32,"C, R",70.0,79.6,23.04,NaN,38.0,68.0,342.00,NaN,64.766357,55.558315,73.654629,45.994624,104.297649,77.783372,0.542857,0.845157,0.040247,NaN,7.91,10.384,2.152064,17.148571,3.150238,78.89,7.00,0.988,6.8e+06,22.0,8.297073
ELIASPETTERSSON,NaN,NaN,nan nan,NaN,VAN,21,C,68.0,69.5,2.25,NaN,65.0,65.0,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.955882,0.935688,0.000408,NaN,12.89,12.280,0.372100,18.531127,3.846324,84.88,8.71,1.033,925000,NaN,8.254285


In [30]:
# TODO: Update csv name
nhl[~(nhl.index == 'NANNAN') & (nhl.POS == 'D')].to_csv(os.path.join(data_folder, 'nhl_def_2021.csv'), sep=';')
nhl[~(nhl.index == 'NANNAN') & (nhl.POS != 'D')].to_csv(os.path.join(data_folder, 'nhl_att_2021.csv'), sep=';')

## 8. Rookies
---

* All datasets were copy/pasted from various sources on the internet.
* Possible sources: hockeynews, futureconsideration, hockeyprospect, habseyeontheprize, hockeywriter, sportingnews, dobberhockey

In [129]:
# Read datasets and standardize index

agg = pd.read_csv('../data/aggregate-rookie-2020.csv', header=None, sep='\t')
agg = agg.reset_index()
agg['Name'] = agg.apply(lambda player: re.sub(r'\W+', '', f'{player[0]}').upper(), axis=1)
agg = agg.set_index('Name')

bl = pd.read_csv('../data/bleacher-rookie-2020.csv', header=None, sep=',')
bl = bl.reset_index()
bl['Name'] = bl.apply(lambda player: re.sub(r'\W+', '', f'{player[0]}').upper(), axis=1)
bl = bl.set_index('Name')

thw = pd.read_csv('../data/thehockeywriters-rookie-2020.csv', header=None, sep=',')
thw = thw.reset_index()
thw['Name'] = thw.apply(lambda player: re.sub(r'\W+', '', f'{player[0]}').upper(), axis=1)
thw = thw.set_index('Name')

dha = pd.read_csv('../data/dobberhockey-att-rookie-2020.csv', header=None, sep=',')
dhd = pd.read_csv('../data/dobberhockey-def-rookie-2020.csv', header=None, sep=',')
dh = pd.concat([dha, dhd])
dh = dh.reset_index()
dh['Name'] = dh.apply(lambda player: re.sub(r'\W+', '', f'{player[0]}').upper(), axis=1)
dh = dh.set_index('Name')

In [130]:
# Join datasets
tmp = dh.join(thw, lsuffix='_dh', rsuffix='_thw', how='outer')
tmp = tmp.join(bl, rsuffix='_bl', how='outer')
rookies = tmp.join(agg, rsuffix='_agg', how='outer')
rookies.columns = ['index_dh', '0_dh', 'pos', 'style', 'index_thw', '0_thw', 'index_bl', 0, 'index_agg', '0_agg']

# Keep only ranks and fill for missing names in datasets
# rookies = rookies.filter(regex=("^index*|^2_*|^0_dh$|^0_sn$|^0$"))

# Fill score of players missing in a list by max position
rookies[['index_dh', 'index_thw', 'index_bl', 'index_agg']] = rookies[['index_dh', 'index_thw', 'index_bl', 'index_agg']].fillna(rookies.max(axis=0))

In [131]:
# Get score and save results
# TODO: change rookie filename
rookies['score'] = rookies.filter(regex=("^index*")).sum(axis=1)
rookies.columns = ['DobberHockey', '0_dh', 'Pos', 'Style', 'TheHockeyWriters', '0_thw', 'BleacherReport', 0, 'Aggregate', '0_agg', 'score']
rookies.to_csv(os.path.join(data_folder, 'rookies-2021.csv'))